In [ ]:
import pandas as pd

# load the Shanghai data
file_path = 'The Shanghai data.xlsx'
df_swimming_age_filtered = pd.read_excel(file_path)

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Assume df_swimming_age_filtered is your DataFrame that includes all independent and control variable columns
# self_vars is a list of the names of independent variables, covars is a list of the names of control variables
self_vars = [
    "Height (cm)", "Weight (kg)", "Calf circumference (cm)", "Upper limb length (cm)",
    "Thigh circumference (cm)", "Lower limb length (cm)", "Pelvis width (cm)",
    "Subscapular skinfold (mm)", "Abdominal skinfold (mm)", "Triceps skinfold (mm)",
    "Shoulder width (cm)", "Achilles tendon length (cm)", "Waist circumference (cm)",
    "Arm span (cm)", "Sitting height (cm)", "Calf length (cm)", "Ankle circumference (cm)",
    "Chest circumference (cm)", "Hemoglobin (g/l)", "Lung capacity (ml)",
    "Systolic pressure (mmHg)", "Diastolic pressure (mmHg)"
]
covars = ["Age", "Sex", "interaction_term"]

# Iterate through each independent variable, fit it with a linear model, and calculate the residuals
for var in self_vars:
    # Get the data for the independent variable and control variables
    X = df_swimming_age_filtered[covars]
    y = df_swimming_age_filtered[var]
    
    # Perform linear fitting using LinearRegression
    model = LinearRegression()
    model.fit(X, y)
    
    # Calculate the fitted values and residuals
    fitted_values = model.predict(X)
    residuals = y - fitted_values
    
    # Update the original column with residuals
    df_swimming_age_filtered[var] = residuals

# Output the processed DataFrame to examine the updated independent variable columns
print(df_swimming_age_filtered.head(6))

In [ ]:
import pandas as pd

# Filter out the rows with ranking of 1 and ranking of 0 separately
df_rank_1 = df_swimming_age_filtered[df_swimming_age_filtered['Rank'] == 1]
df_rank_0 = df_swimming_age_filtered[df_swimming_age_filtered['Rank'] == 0]

# Get the number of rows with ranking of 0 to determine the number of rows with ranking of 1 to be sampled
num_rank_0 = df_rank_0.shape[0]

# Sample (with replacement) from the rows with ranking of 1 to obtain a number of samples equal to the number of rows with ranking of 0
df_rank_1_sampled = df_rank_1.sample(n=num_rank_0, replace=True, random_state=42)

# Combine the resampled rows with ranking of 1 with the rows with ranking of 0
df_balanced = pd.concat([df_rank_1_sampled, df_rank_0]).reset_index(drop=True)

# Output the combined data
print(df_balanced)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assume df_balanced is the DataFrame you want to split

# Split the DataFrame into training and test sets with a ratio of 70-30
train_set, test_set = train_test_split(df_balanced, test_size=0.30, stratify=df_balanced[['Rank', 'Age', 'Sex']], random_state=42)

# Save the data to different sheets in an Excel file
with pd.ExcelWriter('Model_Development_Data.xlsx') as writer:
    train_set.to_excel(writer, sheet_name='training', index=False)
    test_set.to_excel(writer, sheet_name='test', index=False)